## Import libraries

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import datetime

## Crawling

`url = "https://giavang.pnj.com.vn/history?gold_history_day={day_here}&gold_history_month={month_here}&gold_history_year={year_here}"`
- Using `requests` to get response of url . We will crawling the nearest 90 days gold prices.
- Try to find what element where the price in then take value from it : **div.portlet-body[3]/table/tbody/tr/td**
- Save values to a dict then using `pandas` convert to a DataFrame 
- Save it to **.csv**** file using `to_csv()` method.



### Get the nearest 90 days

In [17]:
# Take the nearest 5 days
res = []
today = datetime.date.today()
for i in range(5):
    res.append((today - datetime.timedelta(days=i)).strftime('%Y-%m-%d'))

print(res)

['2024-06-18', '2024-06-17', '2024-06-16', '2024-06-15', '2024-06-14']


### Crawling from giavang.pnj.com.vn

In [30]:
dict = {
    'type': [],
    'buy': [],
    'sell': [],
    'update': []
}

for i in range(180):
    cur_date = (today - datetime.timedelta(days=i)).strftime('%Y-%m-%d')
    print(cur_date)
    year,month,day = cur_date.split('-')
    url = f"https://giavang.pnj.com.vn/history?gold_history_day={day}&gold_history_month={month}&gold_history_year={year}"
    
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, 'html.parser')

    # div.portlet-body[3]/table/tbody/tr/td
    divs = soup.find_all("div", class_="portlet-body")
    gold_price_hcm = divs[3]
    
    table = gold_price_hcm.find("table")

    tbody = table.find("tbody")

    trs = tbody.find_all("tr")[1:]


    type = 'PNJ'
    for tr in trs:
        tds = tr.find_all("td")
        buy = tds[0].text
        sell = tds[1].text
        time = tds[2].text
        if buy == 'PNJ':
            buy = tds[1].text
            sell = tds[2].text
            time = tds[3].text
        if buy == 'SJC':
            buy = tds[1].text
            sell = tds[2].text
            time = tds[3].text
            type = 'SJC'
        dict['type'].append(type)
        dict['buy'].append(sell)
        dict['sell'].append(buy)
        dict['update'].append(time)


2024-06-18
2024-06-17
2024-06-16
2024-06-15
2024-06-14
2024-06-13
2024-06-12
2024-06-11
2024-06-10
2024-06-09
2024-06-08
2024-06-07
2024-06-06
2024-06-05
2024-06-04
2024-06-03
2024-06-02
2024-06-01
2024-05-31
2024-05-30
2024-05-29
2024-05-28
2024-05-27
2024-05-26
2024-05-25
2024-05-24
2024-05-23
2024-05-22
2024-05-21
2024-05-20
2024-05-19
2024-05-18
2024-05-17
2024-05-16
2024-05-15
2024-05-14
2024-05-13
2024-05-12
2024-05-11
2024-05-10
2024-05-09
2024-05-08
2024-05-07
2024-05-06
2024-05-05
2024-05-04
2024-05-03
2024-05-02
2024-05-01
2024-04-30
2024-04-29
2024-04-28
2024-04-27
2024-04-26
2024-04-25
2024-04-24
2024-04-23
2024-04-22
2024-04-21
2024-04-20
2024-04-19
2024-04-18
2024-04-17
2024-04-16
2024-04-15
2024-04-14
2024-04-13
2024-04-12
2024-04-11
2024-04-10
2024-04-09
2024-04-08
2024-04-07
2024-04-06
2024-04-05
2024-04-04
2024-04-03
2024-04-02
2024-04-01
2024-03-31
2024-03-30
2024-03-29
2024-03-28
2024-03-27
2024-03-26
2024-03-25
2024-03-24
2024-03-23
2024-03-22
2024-03-21
2024-03-20

### Save it to csv

In [32]:
df = pd.DataFrame(dict)

# take a overview of the data
df.head()

,type,buy,sell,update
0,PNJ,75.000,73.300,18/06/2024 08:09:01
1,SJC,76.980,74.980,18/06/2024 08:09:01
2,PNJ,75.000,73.300,17/06/2024 08:13:04
3,SJC,76.980,74.980,17/06/2024 08:13:04
4,PNJ,75.100,73.400,15/06/2024 08:09:01


In [33]:
df.shape

(1092, 4)

In [34]:
df.to_csv('six_months_gold_price.csv', index=False)